# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
cities = pd.read_csv("cities.csv", encoding="utf-8")

cities= cities.rename(columns={'Unnamed: 0': 'City_ID','Max Temp': 'max_temp'})

cities.head()

,City_ID,City,Lat,Lng,max_temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,0,pamanukan,-6.2842,107.8106,77.86,89,99,3.71,ID,1617062380
1,1,tuktoyaktuk,69.4541,-133.0374,-7.60,77,20,17.27,CA,1617062149
2,2,hobart,-42.8794,147.3294,61.00,58,0,12.66,AU,1617061791
3,3,knysna,-34.0363,23.0471,57.20,94,0,3.44,ZA,1617062380
4,4,bluff,-46.6000,168.3333,60.01,88,100,1.01,NZ,1617061995


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
gmaps.configure(api_key=g_key)


In [20]:
lng_lat = cities[["Lat","Lng"]]
hum = cities['Humidity'].astype(float)

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(lng_lat, weights=hum, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
cities_fil_df = cities.loc[(cities['Wind_Speed'] <= 10) & (cities['Cloudiness'] == 0) &
                          (cities['max_temp'] > 59) & (cities['max_temp'] <=80)].dropna()

cities_fil_df



,City_ID,City,Lat,Lng,max_temp,Humidity,Cloudiness,Wind_Speed,Country,Date
68,68,saint-philippe,-21.3585,55.7679,77.00,73,0,6.91,RE,1617061990
110,112,dinanagar,32.1500,75.4667,68.22,31,0,6.26,IN,1617062528
121,123,tete,-16.1564,33.5867,75.20,73,0,6.91,MZ,1617062531
122,124,cape town,-33.9258,18.4232,64.00,88,0,8.05,ZA,1617062274
142,146,chokwe,-24.5333,32.9833,63.82,77,0,3.44,MZ,1617062537
233,239,presidencia roque saenz pena,-26.7852,-60.4388,69.01,82,0,2.64,AR,1617062682
284,292,saint-joseph,-21.3667,55.6167,77.00,73,0,6.91,RE,1617062756
308,317,paharpur,32.1038,70.9724,73.09,17,0,7.11,PK,1617062824
366,377,inhambane,-23.8650,35.3833,68.47,70,0,4.52,MZ,1617062900
374,385,caravelas,-17.7125,-39.2481,78.21,73,0,7.94,BR,1617062903


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df = cities_fil_df.loc[:,['City','Country','Lat','Lng']]

hotel_df.reset_index(inplace=True)


hotel_df["Hotel Name"] = ''

hotel_df.head()

,index,City,Country,Lat,Lng,Hotel Name
0,68,saint-philippe,RE,-21.3585,55.7679,
1,110,dinanagar,IN,32.1500,75.4667,
2,121,tete,MZ,-16.1564,33.5867,
3,122,cape town,ZA,-33.9258,18.4232,
4,142,chokwe,MZ,-24.5333,32.9833,


In [23]:
hotellist = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "keyword": "hotel",
        "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    response = requested.json()
    try:
        hotellist.append(response['results'][0]['name'])
    except:
        hotellist.append("")
        
hotel_df["Hotel Name"] = hotellist
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)

hotel_df.head(50)

,index,City,Country,Lat,Lng,Hotel Name
0,68,saint-philippe,RE,-21.3585,55.7679,Les Embruns Du Baril
2,121,tete,MZ,-16.1564,33.5867,Baobab Hotel Tete
3,122,cape town,ZA,-33.9258,18.4232,Atlanticview Cape Town Boutique Hotel
4,142,chokwe,MZ,-24.5333,32.9833,Hotel Sorriso de Caridade
5,233,presidencia roque saenz pena,AR,-26.7852,-60.4388,Hotel Orel
6,284,saint-joseph,RE,-21.3667,55.6167,"""Plantation Bed and Breakfast"
7,308,paharpur,PK,32.1038,70.9724,salem .T. Hotal
8,366,inhambane,MZ,-23.8650,35.3833,Sentidos Beach Retreat
9,374,caravelas,BR,-17.7125,-39.2481,Pousada Liberdade
10,390,villa carlos paz,AR,-31.4241,-64.4978,Piazza Hotel


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)



# Display figure
fig

Figure(layout=FigureLayout(height='420px'))